# Kalman Filter
> Implementation of Kalman filters using pytorch and parameter optimizations with gradient descend

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
#| default_exp kalman.filter

In [ ]:
#| export
from fastcore.test import *
from fastcore.basics import *
from meteo_imp.utils import *
from meteo_imp.gaussian import *
from meteo_imp.data_preparation import MeteoDataTest
from typing import *
from functools import partial

import numpy as np
import pandas as pd
import torch
from torch import Tensor
from torch.distributions import MultivariateNormal

## Introduction

The models uses a latent state variable $x$ that is modelled over time, to impute gaps in $y$

The assumption of the model is that the state variable at time $x_t$ depends only on the last state $x_{t-1}$ and not on the previous states.

### Equations

The equations of the model are:

$$\begin{align} p(x_t | x_{t-1}) & = \mathcal{N}(Ax_{t-1} + b, Q) \\
p(y_t | x_t) & = \mathcal{N}(Hx_t + d, R) \end{align}$$


where:

- $A$ is the `A`
- $b$ is the `bset`
- $Q$ is the `Q`
- $H$ is the `obs_trans` 
- $d$ is the `d`
- $R$ is the `R`

in addition the model has also the parameters of the initial state that are used to initialize the filter:

- `m0`
- `P0`

The Kalman filter has 3 steps:

- filter (updating the state at time t with observations till time t-1)
- update (update the state at time t using the observation at time t)
- smooth (update the state using the observations at time t+1)

In case of missing data the update step is skipped.

After smoothing the missing data at time t ($y_t$) can be imputed from the state ($x_t$) using this formula:
$$p(y_t|x_t) = \mathcal{N}(Hx_t + d, R + HP^s_tH^T)$$

The Kalman Filter is an algorithm designed to estimate $P(x_t | y_{0:t})$.  As all state transitions and obss are linear with Gaussian distributed noise, these distributions can be represented exactly as Gaussian distributions with mean `ms[t]` and covs `Ps[t]`.
Similarly, the Kalman Smoother is an algorithm designed to estimate $P(x_t | y_{0:t-1})$



# Kalman Filter Base

TODO: fill nans with 0 for all data

In [ ]:
#| export
def _add_batch_dim(x):
    """make x 3 dimensional by adding empty dims in the correct place"""
    if x.dim() == 1: return x.unsqueeze(0).unsqueeze(-1)
    elif x.dim() == 2: return x.unsqueeze(0)
    else: return x

In [ ]:
show_as_row(_add_batch_dim(torch.ones(2)), _add_batch_dim(torch.ones(2,2)), _add_batch_dim(torch.ones(2,2,2)))

In [ ]:
show_as_row(_add_batch_dim(torch.ones(2)).shape, _add_batch_dim(torch.ones(2,2)).shape, _add_batch_dim(torch.ones(2,2,2)).shape)

In [ ]:
#| export
class KalmanFilterBase(torch.nn.Module):
    """Base class for handling Kalman Filter implementation in PyTorch"""
    
    params_constr = {
        #name constraint
        'A':  None        ,
        'b':  None        ,
        'Q':  DiagPosDef(),
        'B':  None        ,
        'H':  None        ,
        'd':  None        ,
        'R':  DiagPosDef(),
        'm0': None       ,
        'P0': PosDef()   ,
        }
    
    def __init__(self,
            A: Tensor,                             # [n_dim_state,n_dim_state] $A$, state transition matrix 
            H: Tensor,                             # [n_dim_obs, n_dim_state] $H$, observation matrix
            B: Tensor,                             # [n_dim_state, n_dim_contr] $B$ control matrix
            Q: Tensor,                             # [n_dim_state, n_dim_state] $Q$, state trans covariance matrix
            R: Tensor,                             # [n_dim_obs, n_dim_obs] $R$, observations covariance matrix
            b: Tensor,                             # [n_dim_state] $b$, state transition offset
            d: Tensor,                             # [n_dim_obs] $d$, observations offset
            m0: Tensor,                            # [n_dim_state] $m_0$
            P0: Tensor,                            # [n_dim_state, n_dim_state] $P_0$
    
            n_dim_state: int = None,               # Number of dimensions for state - default infered from parameters
            n_dim_obs: int = None,                 # Number of dimensions for observations - default  infered from parameters
            n_dim_contr: int = None,               # Number of dimensions for control - default infered from parameters
            
            var_names: Iterable[str]|None = None,  # Names of variables for printing 
            contr_names: Iterable[str]|None = None,# Names of control variables for printing
    
            cov_checker: CheckPosDef|None = None,  # Check covariance at every step
            use_conditional: bool = True,          # Use conditional distribution for gaps that don't have all variables missing
            use_control: bool = True,              # Use the control in the filter
            use_smooth: bool = True,               # Use smoother for predictions (otherwise is filter only)
                ):
        
        super().__init__()
        store_attr("var_names, contr_names, use_conditional, use_control, use_smooth, cov_checker")
        
        self._check_params(A, H, B, Q, R, b, d, m0, P0, n_dim_state, n_dim_obs, n_dim_contr)
        self._init_params(A=A, H=H, B=B, Q=Q, R=R, b=b, d=d, m0=m0, P0=P0)

    
    def _check_params(self, A, H, B, Q, R, b, d, m0, P0, n_dim_state, n_dim_obs, n_dim_contr):
        """Checks that the parameters are dimensions are consistent and sets n_dim"""
        self.n_dim_state = determine_dimensionality(
            [(A, array2d, -2),
             (b, array1d, -1),
             (Q, array2d, -2),
             (m0, array1d, -1),
             (P0, array2d, -2),
             (H, array2d, -1)],
            n_dim_state
        )
        self.n_dim_obs = determine_dimensionality(
            [(H, array2d, -2),
             (d, array1d, -1),
             (R, array2d, -2)],
            n_dim_obs
        )
        
        self.n_dim_contr = determine_dimensionality([(B, array2d, -1)], n_dim_contr)
        
        
    def _init_params(self, **params):
        for name, value in params.items():
            value = _add_batch_dim(value)
            if (constraint := self.params_constr[name]) is not None:
                name, value = self._init_constraint(name, value, constraint)
            self._init_param(name, value, train=True)    
    
    def _init_param(self, param_name, value, train):
        self.register_parameter(param_name, torch.nn.Parameter(value, requires_grad=train))
    
    ### === Constraints utils
    def _init_constraint(self, param_name, value, constraint):
        name = f"{param_name}_raw"
        value = constraint.inverse_transform(value)
        setattr(self, param_name + "_constraint", constraint)
        return name, value
    
    def _get_constraint(self, param_name):
        """get the original value"""
        constraint = getattr(self, param_name + "_constraint")
        raw_value = getattr(self, f"{param_name}_raw")
        return constraint.transform(raw_value)

    def _set_constraint(self, value, param_name, train=True):
            """set the transformed value"""
            constraint = getattr(self, param_name + "_constraint")
            raw_value = constraint.inverse_transform(value)
            self._init_param(f"{param_name}_raw", raw_value, train)
            
               
    @property
    def Q_C(self):
        "Cholesky factor of Q"
        return torch.diag_embed(self.Q_raw, dim1=-2, dim2=-1)
    @property
    def Q(self): return self._get_constraint('Q')
    @Q.setter
    def Q(self, value): self._set_constraint(value, 'Q')

    @property
    def R_C(self):
        "Cholesky factor of R"
        return torch.diag_embed(self.R_raw, dim1=-2, dim2=-1)
    @property
    def R(self): return self._get_constraint('R')
    @R.setter
    def R(self, value): self._set_constraint(value, 'R')
    
    @property
    def P0_C(self):
        "Cholesky factor of P0"
        return self.P0_raw
    @property
    def P0(self): return self._get_constraint('P0')
    @P0.setter
    def P0(self, value): self._set_constraint(value, 'P0')


    ### === Utility Func    
    def _parse_obs(self, obs, mask, control):
        """maybe get mask from `nan`"""
        # if mask is None: mask = ~torch.isnan(obs)
        return _add_batch_dim(obs), _add_batch_dim(mask), _add_batch_dim(control)
    def __repr__(self):
        return f"""Kalman Filter
        N dim obs: {self.n_dim_obs},
        N dim state: {self.n_dim_state},
        N dim contr: {self.n_dim_contr}"""

### Constructors

Giving all the parameters manually to the `KalmanFilterBase` init method is not convenient, hence we are having some methods that help initize the class

#### Random parameters

In [ ]:
#| export
@patch(cls_method=True)
def init_random(cls: KalmanFilterBase, n_dim_obs, n_dim_state, n_dim_contr, dtype=torch.float64, **kwargs):
    """kalman filter with random parameters"""
    return cls(
        A  = torch.rand(n_dim_state, n_dim_state, dtype=dtype),
        b  = torch.rand(n_dim_state, dtype=dtype),        
        Q  = to_diagposdef(torch.rand(n_dim_state, n_dim_state, dtype=dtype)),        
        B  = torch.rand(n_dim_state, n_dim_contr, dtype=dtype),
        H  = torch.rand(n_dim_obs, n_dim_state, dtype=dtype),
        d  = torch.rand(n_dim_obs, dtype=dtype),          
        R  = to_diagposdef(torch.rand(n_dim_obs, n_dim_obs, dtype=dtype)),            
        m0 = torch.rand(n_dim_state, dtype=dtype),        
        P0 = to_posdef(torch.rand(n_dim_state, n_dim_state, dtype=dtype)),
        **kwargs) 
        

In [ ]:
kB = KalmanFilterBase.init_random(3,4, 3, dtype=torch.float64)
kB

Kalman Filter
        N dim obs: 3,
        N dim state: 4,
        N dim contr: 3

In [ ]:
test_close(kB.Q_C @ kB.Q_C.mT, kB.Q)

In [ ]:
kB.P0 = to_posdef(torch.rand(1,3,3))

check that assigment works :)

In [ ]:
kB.P0 = to_posdef(torch.rand(4, 4, dtype=torch.float64))

In [ ]:
kB.P0_C

Parameter containing:
tensor([[ 1.2842,  0.0000,  0.0000,  0.0000],
        [ 1.1986,  0.4556,  0.0000,  0.0000],
        [ 1.1782,  0.2900,  0.3007,  0.0000],
        [ 1.1862,  0.1961, -0.2735,  0.2186]], dtype=torch.float64,
       requires_grad=True)

In [ ]:
kB = KalmanFilterBase.init_random(3,4, 3, dtype=torch.float64)
kB

Kalman Filter
        N dim obs: 3,
        N dim state: 4,
        N dim contr: 3

In [ ]:
list(kB.named_parameters())

[('A',
  Parameter containing:
  tensor([[[0.5119, 0.6876, 0.4421, 0.8566],
           [0.4001, 0.1293, 0.8637, 0.1677],
           [0.4323, 0.8422, 0.9664, 0.0836],
           [0.4376, 0.9593, 0.4810, 0.1568]]], dtype=torch.float64,
         requires_grad=True)),
 ('H',
  Parameter containing:
  tensor([[[0.4003, 0.9099, 0.4019, 0.8489],
           [0.6460, 0.6981, 0.6625, 0.4510],
           [0.2156, 0.3109, 0.7897, 0.7239]]], dtype=torch.float64,
         requires_grad=True)),
 ('B',
  Parameter containing:
  tensor([[[0.0400, 0.2521, 0.7528],
           [0.7491, 0.3675, 0.1329],
           [0.0968, 0.7927, 0.1157],
           [0.5598, 0.5978, 0.2464]]], dtype=torch.float64, requires_grad=True)),
 ('Q_raw',
  Parameter containing:
  tensor([[0.1064, 0.1926, 0.7552, 0.7606]], dtype=torch.float64,
         requires_grad=True)),
 ('R_raw',
  Parameter containing:
  tensor([[0.7597, 0.5292, 0.7048]], dtype=torch.float64, requires_grad=True)),
 ('b',
  Parameter containing:
  tensor([[[0

In [ ]:
kB.state_dict()

OrderedDict([('A',
              tensor([[[0.5119, 0.6876, 0.4421, 0.8566],
                       [0.4001, 0.1293, 0.8637, 0.1677],
                       [0.4323, 0.8422, 0.9664, 0.0836],
                       [0.4376, 0.9593, 0.4810, 0.1568]]], dtype=torch.float64)),
             ('H',
              tensor([[[0.4003, 0.9099, 0.4019, 0.8489],
                       [0.6460, 0.6981, 0.6625, 0.4510],
                       [0.2156, 0.3109, 0.7897, 0.7239]]], dtype=torch.float64)),
             ('B',
              tensor([[[0.0400, 0.2521, 0.7528],
                       [0.7491, 0.3675, 0.1329],
                       [0.0968, 0.7927, 0.1157],
                       [0.5598, 0.5978, 0.2464]]], dtype=torch.float64)),
             ('Q_raw',
              tensor([[0.1064, 0.1926, 0.7552, 0.7606]], dtype=torch.float64)),
             ('R_raw',
              tensor([[0.7597, 0.5292, 0.7048]], dtype=torch.float64)),
             ('b',
              tensor([[[0.5072],
                       

### Get Info

In [ ]:
#| export
@patch
def get_info(self: KalmanFilterBase):
    out = {}
    var_names = ifnone(self.var_names, [f"y_{i}" for i in range(self.n_dim_obs)])
    latent_names = [f"x_{i}" for i in range(self.n_dim_state)]
    contr_names = ifnone(self.contr_names, [f"c_{i}" for i in range(self.n_dim_contr)])
    out['$A$'] = array2df(self.A[0] , latent_names, latent_names, 'state')
    out['$Q$']    = array2df(self.Q[0] , latent_names, latent_names, 'state')
    out['$b$']        = array2df(self.b[0] , latent_names, ['offset'],   'state')
    out['$H$']   = array2df(self.H[0] , var_names,    latent_names, 'variable')
    out['$R$']      = array2df(self.R[0] , var_names,    var_names,    'variable')
    out['$d$']          = array2df(self.d[0] , var_names,    ['offset'],   'variable')
    out['$B$'] = array2df(self.B[0] , latent_names, contr_names,  'state')
    out['$m_0$']  = array2df(self.m0[0], latent_names, ['mean'],     'state')
    out['$P_0$']   = array2df(self.P0[0], latent_names, latent_names, 'state')

    return out

In [ ]:
#| export
@patch
def _repr_html_(self: KalmanFilterBase):
    title = f"Kalman Filter ({self.n_dim_obs} obs, {self.n_dim_state} state, {self.n_dim_contr} contr)"
    return row_dfs(self.get_info(), title , hide_idx=True)

In [ ]:
kB

Kalman Filter
        N dim obs: 3,
        N dim state: 4,
        N dim contr: 3

### Test data

In [ ]:
#| exporti
def get_test_data(n_obs = 10, n_dim_obs=3, n_dim_contr = 3, p_missing=.3, bs=2, dtype=torch.float64, device='cpu'):
    data = torch.rand(bs, n_obs, n_dim_obs, dtype=dtype, device=device)
    mask = torch.rand(bs, n_obs, n_dim_obs, device=device) > p_missing
    control = torch.rand(bs, n_obs, n_dim_contr, dtype=dtype, device=device)
    data[~mask] = torch.nan # ensure that the missing data cannot be used
    return data, mask, control

In [ ]:
reset_seed()
data, mask, control = get_test_data()
show_as_row(data, mask, control)

# Standard Kalman Filter

In [ ]:
#| export
class KalmanFilter(KalmanFilterBase):
    pass

In [ ]:
k = KalmanFilter.init_random(3,4,3)

## Filter

### Filter predict

Probability of state at time `t` given state a time `t-1` 

$p(x_t) = \mathcal{N}(x_t; m_t^-, P_t^-)$ where:

- predicted state mean: $m_t^- = Am_{t-1} + B c_t + b$  

- predicted state covariance: $P_t^- = AP_{t-1}A^T + Q$

In [ ]:
A, Q, b, B, m_pr,P_pr= (k.A, k.Q, k.b, k.B,torch.concat([k.m0]*2), torch.concat([k.P0]*2))

In [ ]:
m_pr.shape, P_pr.shape, A.shape

(torch.Size([2, 4, 1]), torch.Size([2, 4, 4]), torch.Size([1, 4, 4]))

#### Covariance

implement $P_t^- = AP_{t-1}A^T + Q$

In [ ]:
#| export
def _filter_predict_cov_stand(A, Q, P_pr):
    """Standard - Kalman Filter predict covariance"""
    return A @ P_pr @ A.mT + Q

In [ ]:
P_m = _filter_predict_cov_stand(A, Q, P_pr)
P_m

tensor([[[4.8187, 4.4353, 4.9715, 4.9817],
         [4.4353, 5.3111, 5.1082, 5.1311],
         [4.9715, 5.1082, 5.7240, 5.7099],
         [4.9817, 5.1311, 5.7099, 6.3261]],

        [[4.8187, 4.4353, 4.9715, 4.9817],
         [4.4353, 5.3111, 5.1082, 5.1311],
         [4.9715, 5.1082, 5.7240, 5.7099],
         [4.9817, 5.1311, 5.7099, 6.3261]]], dtype=torch.float64,
       grad_fn=<AddBackward0>)

#### Mean

In [ ]:
#| export
def _filter_predict_mean(
    A,      # transition matrix
    B,      # control matrix
    b,      # transition offset
    m_pr,   # Mean previous time step $m_{t-1}$
    control, # control variable
):
    return A @ m_pr + B @ control + b

#### Predict

In [ ]:
#| export
def _filter_predict(A,
                    Q,
                    b,
                    B, #[n_dim_state, n_dim_contr]
                    m_pr,
                    P_pr,
                    control, #[n_batches, n_dim_contr]
                    ):
    """Calculate the state at time `t` given the state at time `t-1`"""
    m_m = _filter_predict_mean(A, B, b, m_pr, control)
    P_m = _filter_predict_cov_stand(A, Q, P_pr)
    return (m_m, P_m)

In [ ]:
#| export
def _filter_predict_mean(
    A,      # transition matrix
    B,      # control matrix
    b,      # transition offset
    m_pr,   # Mean previous time step $m_{t-1}$
    control, # control variable
):
    return A @ m_pr + B @ control + b

In [ ]:
#| export
def unsqueeze_iter(*args, dim): return list(map(partial(torch.unsqueeze, dim=dim), args))
unsqueeze_first = partial(unsqueeze_iter, dim=0)
unsqueeze_last = partial(unsqueeze_iter, dim=-1)

In [ ]:
#| export
def _filter_predict(A,
                    Q,
                    b,
                    B, #[n_dim_state, n_dim_contr]
                    m_pr,
                    P_pr,
                    control, #[n_batches, n_dim_contr]
                    ):
    """Calculate the state at time `t` given the state at time `t-1`"""
    m_m = _filter_predict_mean(A, B, b, m_pr, control)
    P_m = _filter_predict_cov_stand(A, Q, P_pr)
    return (m_m, P_m)

In [ ]:
B.shape

torch.Size([1, 4, 3])

In [ ]:
control.shape

torch.Size([2, 10, 3])

In [ ]:
B[0].shape

torch.Size([4, 3])

In [ ]:
B[0] @ control[0, 0].unsqueeze(-1)

tensor([[0.7217],
        [0.9572],
        [0.5742],
        [1.2138]], dtype=torch.float64, grad_fn=<MmBackward0>)

In [ ]:
m_m, P_m = _filter_predict(
    A, Q, b, B,
    m_pr,P_pr, control[:, 0, :].unsqueeze(-1))

In [ ]:
show_as_row(m_m, P_m)

In [ ]:
(m_m.shape, P_m.shape,)

(torch.Size([2, 4, 1]), torch.Size([2, 4, 4]))

### Filter update

Probability of state at time `t` given the observations at time `t`

$p(x_t|y_t) = \mathcal{N}(x_t; m_t, P_t)$ where:

- predicted obs mean: $z_t = Hm_t^- + d$  

- prediced obs covariance: $S_t = HP_t^-H^T + R$

- kalman gain$K_t = P_t^-H^TS_t^{-1}$ 

- corrected state mean: $m_t = m_t^- + K_t(y_t - z_t)$ 

- corrected state covariance: $P_t = (I-K_tH)P_t^-$ 

if the observation are missing this step is skipped and the corrected state is equal to the predicted state


Need to figure out the Nans for the gradients ...

#### Kalman Gain

Don't compute the inverse of the matrix, but use `cholesky_solve` to invert the matrix

In [ ]:
#| export
def _filter_update_k_gain(H, R,P_m):
    "kalman gain for filter update"
    S = H @ P_m @ H.mT + R
    S_C = torch.linalg.cholesky(S)
    return torch.cholesky_solve(H @ P_m.mT, S_C).mT

In [ ]:
K = _filter_update_k_gain(H, R, P_m)
K

tensor([[[ 0.0307,  0.3311,  0.0291],
         [-0.2621,  0.4961,  0.0624],
         [ 0.0015,  0.4326, -0.0126],
         [ 0.2272,  0.3601, -0.0535]],

        [[ 0.0307,  0.3311,  0.0291],
         [-0.2621,  0.4961,  0.0624],
         [ 0.0015,  0.4326, -0.0126],
         [ 0.2272,  0.3601, -0.0535]]], dtype=torch.float64,
       grad_fn=<TransposeBackward0>)

In [ ]:
test_close(_filter_update_k_gain(H, R, P_m), P_m @ H.mT @ torch.inverse(H @ P_m @ H.mT + R))

#### Covariance

In [ ]:
#| export
def _filter_update_cov(H, K, P_m):
    return (eye_like(P_m) - K @ H) @ P_m

In [ ]:
P = _filter_update_cov(H, K, P_m)
P

tensor([[[ 0.5970, -0.1075,  0.1579, -0.0676],
         [-0.1075,  0.3210, -0.0807, -0.2334],
         [ 0.1579, -0.0807,  0.2325, -0.0442],
         [-0.0676, -0.2334, -0.0442,  0.2359]],

        [[ 0.5970, -0.1075,  0.1579, -0.0676],
         [-0.1075,  0.3210, -0.0807, -0.2334],
         [ 0.1579, -0.0807,  0.2325, -0.0442],
         [-0.0676, -0.2334, -0.0442,  0.2359]]], dtype=torch.float64,
       grad_fn=<UnsafeViewBackward0>)

#### Mean

In [ ]:
z = H @ m_m + d; z
(obs - z)

tensor([[[-3.3343],
         [    nan],
         [    nan]],

        [[-2.4181],
         [-5.0754],
         [-3.9145]]], dtype=torch.float64, grad_fn=<SubBackward0>)

In [ ]:
#| export
def _filter_update_mean(H, d, K, m_m, y):
    z = H @ m_m + d
    return m_m + K @ (y - z)

In [ ]:
m = _filter_update_mean(H, d, K, m_m, obs)
m

tensor([[[    nan],
         [    nan],
         [    nan],
         [    nan]],

        [[ 0.2687],
         [-0.5319],
         [-0.6131],
         [ 0.6636]]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [ ]:
def _filter_update(
    H, # [1, n_dim_obs, n_dim_state]
    d, # [1, n_dim_obs, 1]
    R, # [1, n_dim_obs, n_dim_obs]
    m_m, # [n_batches, n_dim_state, 1]
    P_m, # [n_batches, n_dim_state, n_dim_state]
    obs # # [n_batches, n_dim_obs, 1]
) -> Tuple: # Filtered state (mean, covariance) [n_batches, n_dim_state]
    "Filter update state at `t` with obs at `t`"
    K = _filter_update_k_gain(H, R, P_m)
    m = _filter_update_mean(H, d, K, m_m, obs)
    P = _filter_update_cov(H, K, P_m)
    return m, P

In [ ]:
m, P = _filter_update(H, d, R, m_m, P_m, obs)
show_as_row(m, P)
m.shape, P.shape

(torch.Size([2, 4, 1]), torch.Size([2, 4, 4]))

there are `nan` in the output because there are `nan` in the observations

The next functions adds the support for missing obsevations by also using a mask

#### Missing observations

If all the observations at time $t$ are missing the correct step is skipped and the filtered state at time $t$ () is the same of the filtered state.

If only some observations are missing a variation of equation can be used.

$y^{ng}_t$ is a vector containing the observations that are not missing at time $t$. 

It can be expressed as a linear transformation of $y_t$

$$ y^{ng}_t = My_t$$

where $M$ is a mask matrix that is used to select the subset of $y_t$ that is observed. $M \in \mathbb{R}^{n_{ng} \times n}$ and is made of columns which are made of all zeros but for an entry 1 at row corresponding to the non-missing observation.
hence:

$$ p(y^{ng}_t) = \mathcal{N}(M\mu_{y_t},  M\Sigma_{y_t}M^T)$$

from which you can derive

$$ p(y^{ng}_t|x_t) = p(MHx_t + Mb, MRM^T) $${#eq-filter-correct}

Then the posterior $p(x_t|y_t^{ng})$ can be computed similarly of equation @filter_correct as:

$$ p(x_t|y^{ng}_t) = \mathcal{N}(x_t; m_t, P_t) $${#eq-filter_correct_missing}
    
where:

*  predicted obs mean: $z_t = MHm_t^- + Md$
*  predicted obs covariance: $S_t = MHP_t^-(MH)^T + MRM^T$
*  Kalman gain $K_t = P_t^-(MH)^TS_t^{-1}$
*  corrected state mean: $m_t = m_t^- + K_t(My_t - z_t)$
*  corrected state covariance: $P_t = (I-K_tMH)P_t^-$


##### Details implementation 

For the implementation the matrix multiplication $MH$ can be replaced with `H[m]` where `m` is the mask for the rows for `H` and $MRM^T$ with `R[m][:,m]`

In [ ]:
H, R, d,obs, mm = k.H, k.R, k.d, data[:,0,:].unsqueeze(-1), mask[:,0,:].unsqueeze(-1)

In [ ]:
m = torch.tensor([False,True,True]) # mask batch
M = torch.tensor([[[0,1,0], # mask matrix
                  [0,0,1]]], dtype=torch.float64)
show_as_row(m, M, H, R)

In [ ]:
M @ H, H[:, m]

(tensor([[[0.1679, 0.8635, 0.3753, 0.9760],
          [0.2125, 0.8049, 0.2124, 0.6794]]], dtype=torch.float64,
        grad_fn=<UnsafeViewBackward0>),
 tensor([[[0.1679, 0.8635, 0.3753, 0.9760],
          [0.2125, 0.8049, 0.2124, 0.6794]]], dtype=torch.float64,
        grad_fn=<IndexBackward0>))

In [ ]:
M @ R @ M.mT, R[:,m][:,:,m]

(tensor([[[0.0022, 0.0000],
          [0.0000, 0.9592]]], dtype=torch.float64,
        grad_fn=<UnsafeViewBackward0>),
 tensor([[[0.0022, 0.0000],
          [0.0000, 0.9592]]], dtype=torch.float64, grad_fn=<IndexBackward0>))

By using partially missing observations `_filter_update` cannot be easily batched as the shape of the intermediate variables depends on the number of observed variables. So the idea is to divide the batch in blocks that share the same number of variables missing.

In [ ]:
mask_values, indices = torch.unique(mask[:,1,:], dim=0, return_inverse=True)
mask_values, indices

(tensor([[ True, False,  True],
         [ True,  True, False]]),
 tensor([0, 1]))

##### Update mask

In [ ]:
#| export
def _filter_update_mask(
        H, # [1, n_dim_obs, n_dim_state]
        d, # [1, n_dim_obs, 1]
        R, # [1, n_dim_obs, n_dim_obs]
        m_m, # [n_batches, n_dim_state, 1]
        P_m, # [n_batches, n_dim_state, n_dim_state]
        obs, # [n_batches, n_dim_obs, 1] observations
        mask # [n_dim_obs] mask must be the same across batches
                       ):
    """Update state at time `t` given observations at time `t` assuming that all observations have the same mask"""
    if (~mask).all(): return (m_m, P_m) # all data is missing
    H_m, d_m, R_m, obs_m, = H[:, mask,:], d[:, mask,:], R[:, mask,:][:, :,mask], obs[:, mask] # _m for masked
    return _filter_update(H_m, d_m, R_m, m_m, P_m, obs_m)

In [ ]:
H[:, mm].shape, d[:, mm].shape, R[:, mm][:, :,mm].shape, obs[:, mm].shape

(torch.Size([1, 1, 4]),
 torch.Size([1, 1, 1]),
 torch.Size([1, 1, 1]),
 torch.Size([2, 1, 1]))

In [ ]:
show_as_row(*_filter_update_mask(H, d, R, m_m, P_m, obs, mask[0, 0, :] ))

In [ ]:
m, P = _filter_update_mask(H, d, R, m_m, P_m, obs, mask[0, 0, :] )
m.shape, P.shape

(torch.Size([2, 4, 1]), torch.Size([2, 4, 4]))

##### Update mask batch

In [ ]:
#| export
def _filter_update_mask_batch(
        H, # [1, n_dim_obs, n_dim_state]
        d, # [1, n_dim_obs, 1]
        R, # [1, n_dim_obs, n_dim_obs]
        m_m, # [n_batches, n_dim_state, 1]
        P_m, # [n_batches, n_dim_state, n_dim_state]
        obs, # [n_batches, n_dim_obs, 1] observations
        mask # [n_batches, n_dim_obs] mask must be the same across batches
                       ):
    """Support batches with different masks when update state at time `t` given observations at time `t`"""
    
    ms, Ps= torch.empty_like(m_m), torch.empty_like(P_m)
    
    # find the unique values of the mask and make a sub-batches with it
    mask_values, indices = torch.unique(mask, return_inverse=True, dim=0)  
    for i, mask_v in enumerate(mask_values):
        idx_select = indices == i 
        m, P = _filter_update_mask(
            H, d, R,
            m_m[idx_select], P_m[idx_select],
            obs[idx_select],
            mask_v,
        )
        ms[idx_select], Ps[idx_select] = m, P
    
    return ms, Ps

In [ ]:
m, P = _filter_update_mask_batch(H, d, R, m_m, P_m, obs, mask[:,0,:] )
show_as_row(m, P)
m.shape, P.shape

(torch.Size([2, 4, 1]), torch.Size([2, 4, 4]))

In [ ]:
m.sum().backward(retain_graph=True) # check that pytorch can compute gradients with the whole batch and gradients aren't nan
H.grad

tensor([[[-9.4959e+00, -7.2651e+00,  1.5332e+00, -1.7932e+01],
         [-3.5221e+00,  4.5416e+00,  4.3494e+00, -5.1217e+00],
         [ 3.4425e-01,  2.0932e-03,  2.5141e-01, -1.5584e-01]]],
       dtype=torch.float64)

### Filter

The resursive version of the kalman filter is apperently breaking pytorch gradients calculations so a workaround is needed.
During the loop the states are saved in a python list and then at the end they are combined back into a tensor.
The last line of the function does:

- convert lists to tensors
- correct order dimensions

In [ ]:
#| export
def _times2batch(x):
    """Permutes `x` so that the first dimension is the number of batches and not the times"""
    return x.permute(1,0,-2,-1)

In [ ]:
#| export
@patch
def _filter_all(self: KalmanFilter,
            obs, # [n_batches, ]
            mask,
            control,
            
           ) ->Tuple[ListMNormal, ListMNormal]: # Filtered state, predicted state 
    """Filter observations using kalman filter """
    obs, mask, control = self._parse_obs(obs, mask, control)
    n_obs = obs.shape[-2]
    bs = obs.shape[0]
    # lists are mutable so need to copy them
    m_ms, P_ms, ms, Ps = [[None for _ in range(n_obs)].copy() for _ in range(4)] 

    for t in range(n_obs):
        # Predict
        if t == 0:
            m_ms[t], P_ms[t] = self.m0.expand(bs, -1, -1), self.P0.expand(bs, -1, -1)
        else:
            m_ms[t], P_ms[t] = _filter_predict(self.A, self.Q, self.b, self.B, ms[t - 1], Ps[t - 1], control[:,t,:])
        
        # Update
        ms[t], Ps[t] = _filter_update_mask_batch(self.H, self.d, self.R, m_ms[t], P_ms[t], obs[:,t,:], mask[:,t,:])
        
        if self.cov_checker is not None:
            self.cov_checker.check(P_ms[t], t=t, name="filter_predict")
            self.cov_checker.check(Ps[t], t=t, name="filter_update")
    
    m_ms, P_ms, ms, Ps = list(maps(torch.stack, _times2batch, (m_ms, P_ms, ms, Ps,))) # reorder dimensions and convert to tensor
    return ListMNormal(ms, Ps), ListMNormal(m_ms, P_ms) 

In [ ]:
obs, m0, P0 = data, k.m0, k.P0

In [ ]:
m_ms, P_ms, ms, Ps = _filter(
    A, H, B,
    Q, R,
    b, d,
    m0, P0,
    data, mask, control)

Predictions at time `0` for both batches

In [ ]:
show_as_row(list(map(Self.shape(), (m_ms, P_ms, ms, Ps,))))

In [ ]:
show_as_row(list(map(lambda x:x[0][0], (m_ms, P_ms, ms, Ps,))))

### KalmanFilter method

In [ ]:
#| export
@patch
def _filter_all(self: KalmanFilter, obs, mask, control
               ) ->Tuple[List, List, List, List]: # m_ms, P_ms, ms, Ps
    """ wrapper around `_filter`"""
    obs, mask = self._parse_obs(obs, mask)
    return _filter(
            self.A, self.H,
            self.B if self.use_control else torch.zeros_like(self.B),
            self.Q, self.R,
            self.b, self.d,
            self.m0, self.P0,
            obs, mask, control,
            self.cov_checker
        )

In [ ]:
pred_mean, _, _, _ = k._filter_all(obs, mask, control);

In [ ]:
type(k._filter_all(obs, mask, control))

In [ ]:
pred_mean.sum().backward(retain_graph=True) # it works!

The filter methods wraps `_filter_all` but in addition:

- returns only filtered state
- detach tensors

In [ ]:
#| export
@patch
def filter(self: KalmanFilter,
          obs: Tensor, # [n_timesteps, n_dim_obs] obs for times [0...n_timesteps-1]
          mask: Tensor,  # [n_timesteps, n_dim_obs] obs for times [0...n_timesteps-1]
          control: Tensor, # [n_timesteps, n_dim_contr] control for times [1...n_timesteps-1]
          ) -> ListMNormal: # Filtered state
    """Filter observation"""
    _, _, ms, Ps = self._filter_all(obs, mask, control)
    return ListMNormal(ms.squeeze(-1), Ps)

In [ ]:
filt = k.filter(obs, mask, control)
filt.mean.shape, filt.cov.shape

## Smooth

### Smooth step

compute the probability of the state at time `t` given all the observations

$p(x_t|Y) = \mathcal{N}(x_t; m_t^s, P_t^s)$ where:

- Kalman smoothing gain: $G_t = P_tA^T(P_{t+1}^-)^{-1}$
- smoothed mean: $m_t^s = m_t + G_t(m_{t+1}^s - m_{t+1}^-)$
- smoothed covariance: $P_t^s = P_t + G_t(P_{t+1}^s - P_{t+1}^-)G_t^T$

In [ ]:
#| export
def _smooth_update(A,                # [n_dim_state, n_dim_state]
                   filt_state: MNormal,         # [n_dim_state] filtered state at time `t`
                   pred_state: MNormal,         # [n_dim_state] state before filtering at time `t + 1` (= using the observation until time t)
                   next_smoothed_state: Normal, # [n_dim_state] smoothed state at time  `t+1`
                   cov_checker = CheckPosDef()
                   ) -> MNormal:                # mean and cov of smoothed state at time `t`
    """Correct a pred state with a Kalman Smoother update"""
    kalman_smoothing_gain = filt_state.cov @ A.unsqueeze(0).mT @ torch.inverse(pred_state.cov) # torch.cholesky_inverse(torch.linalg.cholesky(pred_state.cov))

    m_p = filt_state.mean + kalman_smoothing_gain @ (next_smoothed_state.mean - pred_state.mean)
    P_p = filt_state.cov + kalman_smoothing_gain @ (next_smoothed_state.cov - pred_state.cov) @ kalman_smoothing_gain.mT

    cov_checker.check(P_p, caller='smooth_update')
    
    return MNormal(m_p, P_p)

In [ ]:
filt_state, pred_state, next_smoothed_state = [MNormal(m_m, P_m)] * 3 # just for testing

In [ ]:
show_as_row(*_smooth_update(A, MNormal(m_m, P_m), MNormal(m_m, P_m), MNormal(m_m, P_m)))

In [ ]:
show_as_row(*map(Self.shape(), _smooth_update(A, MNormal(m_m, P_m), MNormal(m_m, P_m), MNormal(m_m, P_m))))

### Smooth

In [ ]:
#| export
def _smooth(A, # `[n_dim_state, n_dim_state]`
            filt_state: ListMNormal, # `[n_timesteps, n_dim_state]`
                # `ms[t]` is the state estimate for time t given obs from times `[0...t]`
            pred_state: ListMNormal, # `[n_timesteps, n_dim_state]`
                # `m_ms[t]` is the state estimate for time t given obs from times `[0...t-1]`
            cov_checker = CheckPosDef()
           ) -> ListMNormal: # `[n_timesteps, n_dim_state]` Smoothed state 
    """Apply the Kalman Smoother"""
    x = pred_state.mean # sample for getting tensor properties
    bs, n_timesteps, n_dim_state = x.shape[0], x.shape[1], x.shape[2]

    smoothed_state = ListMNormal(torch.zeros((bs, n_timesteps,n_dim_state,1),             dtype=x.dtype, device=x.device), 
                                 torch.zeros((bs, n_timesteps, n_dim_state,n_dim_state), dtype=x.dtype, device=x.device))
    # For the last timestep cannot use the smoother
    smoothed_state.mean[:,-1,] = filt_state.mean[:,-1]
    smoothed_state.cov[:,-1] = filt_state.cov[:,-1]

    for t in reversed(range(n_timesteps - 1)):
        (smoothed_state.mean[:,t], smoothed_state.cov[:,t]) = (
            _smooth_update(
                A,
                filt_state[:,t],
                pred_state[:,t + 1],
                smoothed_state[:,t+1],
            )
        )
    return smoothed_state

In [ ]:
(m_ms, P_ms, ms, Ps ) = k._filter_all(data, mask, control)
filt_state, pred_state = ListMNormal(ms, Ps), ListMNormal(m_ms, P_ms)

In [ ]:
smooth_state = _smooth(k.A,  filt_state, pred_state)

In [ ]:
show_as_row(smooth_state.mean[0][0], smooth_state.cov[0][0])

In [ ]:
show_as_row(smooth_state.mean.shape, smooth_state.cov.shape)

### KalmanFilter method

In [ ]:
#| export
@patch
def smooth(self: KalmanFilter,
           obs: Tensor,
           mask: Tensor,
           control: Tensor
          ) -> ListMNormal: # `[n_timesteps, n_dim_state]` smoothed state
        
    """Kalman Filter Smoothing"""

    (m_ms, P_ms, ms, Ps) = self._filter_all(obs, mask, control)

    smoothed_state = _smooth(self.A,
                   ListMNormal(ms, Ps), ListMNormal(m_ms, P_ms),
                   self.cov_checker)
    smoothed_state.mean.squeeze_(-1)
    return smoothed_state

In [ ]:
smoothed_state = k.smooth(data, mask, control)

In [ ]:
show_as_row(smoothed_state.mean.shape, smoothed_state.cov.shape)

## Predict

The prediction at time t ($y_t$) are computed rom the state ($x_t$) using this formula:
$$p(y_t|x_t) = \mathcal{N}(Hx_t + d, R + HP^s_tH^T)$$

this works both if the state was filtered or smoother

This add the supports for conditional predictions, which means that at the time (t) when we are making the predictions some of the variables have been actually observed. Since the model prediction is a normal distribution we can condition on the observed values and thus improve the predictions. See `conditional_gaussian`

In order to have conditional predictions that make sense it's not possible to return the full covariance matrix for the predictions but only the standard deviations

In [ ]:
test_m = torch.tensor(
    [[True, True, True,],
    [False, True, True],
    [False, False, False]]
)

In [ ]:
torch.logical_xor(test_m.all(-1), test_m.any(-1))

In [ ]:
A = torch.rand(2,2,3,3)

In [ ]:
(A @ A).shape

predict can be vectorized across both the batch and the timesteps, except for timesteps that require conditional predictions

In [ ]:
#| export
@patch
def _obs_from_state(self: KalmanFilter, state: ListMNormal):

    mean = self.H @ state.mean.unsqueeze(-1) + self.d.unsqueeze(-1)
    cov = self.H @ state.cov @ self.H.mT + self.R
    
    for c in cov: # this is batched and for all timestamps
        self.cov_checker.check(c, caller='predict')
    
    return ListMNormal(mean.squeeze(-1), cov)

In [ ]:
smoothed_state.mean.shape, smoothed_state.cov.shape

In [ ]:
(k.H @ smoothed_state.mean.unsqueeze(-1)).shape

In [ ]:
pred_obs0 = k._obs_from_state(smoothed_state)
pred_obs0.mean.shape

In [ ]:
pred_obs0.cov.shape

In [ ]:
#| export
@patch
def predict(self: KalmanFilter, obs, mask, control, smooth=True):
    """Predicted observations at all times """
    state = self.smooth(obs, mask, control) if smooth else self.filter(obs, mask, control)
    obs, mask = self._parse_obs(obs, mask)
    
    pred_obs = self._obs_from_state(state)
    pred_mean, pred_std = pred_obs.mean, cov2std(pred_obs.cov)
    
    if self.use_conditional:
        # conditional predictions are slow, do only if some obs are missing 
        cond_mask = torch.logical_xor(mask.all(-1), mask.any(-1))

        # this cannot be batched so returns a list
        cond_preds = cond_gaussian_batched(
            pred_obs[cond_mask], obs[cond_mask], mask[cond_mask])
    
        for i, c_pred in enumerate(cond_preds):
            m = ~mask[cond_mask][i]
            pred_mean[cond_mask][i][m] = c_pred.mean
            pred_std [cond_mask][i][m] = cov2std(c_pred.cov)
    
    return ListNormal(pred_mean, pred_std)

In [ ]:
pred = k.predict(data, mask, control)

In [ ]:
pred.mean.shape, pred.std.shape

Gradients ...

In [ ]:
def get_grad_mask(x):
    "filter gradient after sub the masks value with x"
    d = data.clone()
    d[~mask] = x
    k.predict(data, mask, control).mean.sum().backward(retain_graph=True)
    grad = k.R_raw.grad.clone()
    k.zero_grad() 
    return grad

In [ ]:
get_grad_mask(10)

In [ ]:
test_close(get_grad_mask(1), get_grad_mask(10))

In [ ]:
@patch
def predict_times(self: KalmanFilter, times, obs, mask=None, smooth=True, check_args=None):
    """Predicted observations at specific times """
    state = self.smooth(obs, mask, check_args) if smooth else self.filter(obs, mask, check_args)
    obs, mask = self._parse_obs(obs, mask)
    times = array1d(times)
    
    n_timesteps = obs.shape[0]
    n_features = obs.shape[1] if len(obs.shape) > 1 else 1
    
    if times.max() > n_timesteps or times.min() < 0:
        raise ValueError(f"provided times range from {times.min()} to {times.max()}, which is outside allowed range : 0 to {n_timesteps}")

    means = torch.empty((times.shape[0], n_features), dtype=obs.dtype, device=obs.device)
    stds = torch.empty((times.shape[0], n_features), dtype=obs.dtype, device=obs.device) 
    for i, t in enumerate(times):
        mean, std = self._obs_from_state(
            state.mean[t],
            state.cov[t],
            {'t': t, **check_args} if check_args is not None else None
        )
        
        means[i], stds[i] = _get_cond_pred(ListNormal(mean, std), obs[t], mask[t])
    
    return ListNormal(means, stds)  

# Numerical Stable Kalman Filter (Square Root Filter)

In [ ]:
class KalmanFilterSR(KalmanFilter):
    pass

In [ ]:
kSR = KalmanFilterSR.init_random(3,4,3)

## Filter 

### Filter predict

#### Covariance

Implement the numerical stable version of the covariance update

In [ ]:
A, Q, b, B, m_pr,P_pr= (k.A, k.Q, k.b, k.B,torch.concat([k.m0]*2), torch.concat([k.P0]*2))

In [ ]:
Q_C = kSR.Q_C

In [ ]:
_filter_predict_cov_stand(kSR.A, kSR.Q_C @ kSR.Q_C.mT, P_pr)

tensor([[[4.8187, 4.4353, 4.9715, 4.9817],
         [4.4353, 5.3111, 5.1082, 5.1311],
         [4.9715, 5.1082, 5.7240, 5.7099],
         [4.9817, 5.1311, 5.7099, 6.3261]],

        [[4.8187, 4.4353, 4.9715, 4.9817],
         [4.4353, 5.3111, 5.1082, 5.1311],
         [4.9715, 5.1082, 5.7240, 5.7099],
         [4.9817, 5.1311, 5.7099, 6.3261]]], dtype=torch.float64,
       grad_fn=<AddBackward0>)

In [ ]:
P_pr_C = torch.linalg.cholesky(P_pr)

$$W = \begin{bmatrix}AC_{t-1}&C_Q\end{bmatrix}$$

In [ ]:
W = torch.concat([A @ P_pr_C, Q_C.expand_as(P_pr_C)], dim=-1)
W.shape

torch.Size([2, 4, 8])

In [ ]:
U, R = torch.linalg.qr(W.mT) # U to avod name conflicts

In [ ]:
P_m_C = R.mT

In [ ]:
P_m_C

tensor([[[-2.1952,  0.0000,  0.0000,  0.0000],
         [-2.0205,  1.1084,  0.0000,  0.0000],
         [-2.2647,  0.4802,  0.6036,  0.0000],
         [-2.2694,  0.4924,  0.5530, -0.7922]],

        [[-2.1952,  0.0000,  0.0000,  0.0000],
         [-2.0205,  1.1084,  0.0000,  0.0000],
         [-2.2647,  0.4802,  0.6036,  0.0000],
         [-2.2694,  0.4924,  0.5530, -0.7922]]], dtype=torch.float64,
       grad_fn=<TransposeBackward0>)

In [ ]:
P_m_C @ P_m_C.mT

tensor([[[4.8187, 4.4353, 4.9715, 4.9817],
         [4.4353, 5.3111, 5.1082, 5.1311],
         [4.9715, 5.1082, 5.7240, 5.7099],
         [4.9817, 5.1311, 5.7099, 6.3261]],

        [[4.8187, 4.4353, 4.9715, 4.9817],
         [4.4353, 5.3111, 5.1082, 5.1311],
         [4.9715, 5.1082, 5.7240, 5.7099],
         [4.9817, 5.1311, 5.7099, 6.3261]]], dtype=torch.float64,
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
_filter_predict_cov_stand(A, Q_C @ Q_C.mT, P_pr)

tensor([[[4.8187, 4.4353, 4.9715, 4.9817],
         [4.4353, 5.3111, 5.1082, 5.1311],
         [4.9715, 5.1082, 5.7240, 5.7099],
         [4.9817, 5.1311, 5.7099, 6.3261]],

        [[4.8187, 4.4353, 4.9715, 4.9817],
         [4.4353, 5.3111, 5.1082, 5.1311],
         [4.9715, 5.1082, 5.7240, 5.7099],
         [4.9817, 5.1311, 5.7099, 6.3261]]], dtype=torch.float64,
       grad_fn=<AddBackward0>)

In [ ]:
#| export
def _filter_predict_cov_SR(A, # transition covariance $A_t$
                        Q_C, # Cholesky Factor of transition covariance $Q_t$
                        P_pr_C # Cholesky Factor of previous state covariance $P_{t-1}$
                       ):
    """Numerical stable Kalman filter predict for covariance"""
    W = torch.concat([A @ P_pr_C, Q_C.expand_as(P_pr_C)], dim=-1)
    return torch.linalg.qr(W.mT).R.mT 

In [ ]:
P_m_C = _filter_predict_cov_SR(A, Q_C, P_pr_C)
test_close(P_m_C @ P_m_C.mT, _filter_predict_cov_stand(A, Q_C @ Q_C.mT, P_pr))

In [ ]:
def fuzz_filter_predict_cov_SR(n=10):
    for _ in range(n):
        kSR = KalmanFilterSR.init_random(5,10,4)
        A, Q_C, b, B, m_pr,P_pr = (kSR.A.unsqueeze(0), kSR.Q_C.unsqueeze(0), kSR.b.unsqueeze(-1),
                                                  kSR.B.unsqueeze(0),
                                                  torch.stack([kSR.m0]*2).unsqueeze(-1),
                                                  torch.stack([kSR.P0]*2))
        P_pr_C = torch.linalg.cholesky(P_pr)
        P_m_C = _filter_predict_cov_SR(A, Q_C, P_pr_C)
        test_close(P_m_C @ P_m_C.mT, _filter_predict_cov_stand(A, Q_C @ Q_C.mT, P_pr), eps=5e-13) 

In [ ]:
fuzz_filter_predict_cov_SR()

#### Predict

In [ ]:
#| export
def _filter_predict_SR(A, Q_C, b, B, m_pr, P_pr_C,control) -> Tuple: # predicted state
    """Calculate the state at time `t` given the state at time `t-1`"""
    m_m = _filter_predict_mean(A, B, b, m_pr, control)
    P_m = _filter_predict_cov_SR(A, Q_C, P_pr_C)
    return (m_m, P_m)

### Filter Update

$$M = \begin{bmatrix}R^{T/2} & 0 \\ (C^-)^TH^T & (C^-)^T \end{bmatrix}$$

In [ ]:
R_C = kSR.R_C

In [ ]:
P_pr_C.shape

torch.Size([2, 4, 4])

In [ ]:
P_m_C.shape

torch.Size([2, 4, 4])

In [ ]:
[[R_C.shape,0],
 [(P_m_C.mT @ H.mT).shape, P_m_C.mT.shape]]

[[torch.Size([1, 3, 3]), 0], [torch.Size([2, 4, 3]), torch.Size([2, 4, 4])]]

In [ ]:
M_21 = P_m_C.mT @ H.mT 

In [ ]:
M_21.mT.shape

torch.Size([2, 3, 4])

In [ ]:
M_21.shape[:-2]

torch.Size([2])

In [ ]:
[[R_C.expand(*M_21.shape[:-2], -1, -1).shape, torch.zeros_like(M_21.mT).shape],
 [M_21.shape, P_m_C.mT.shape]]

[[torch.Size([2, 3, 3]), torch.Size([2, 3, 4])],
 [torch.Size([2, 4, 3]), torch.Size([2, 4, 4])]]

In [ ]:
M_1 = torch.cat([R_C.expand(*M_21.shape[:-2], -1, -1), torch.zeros_like(M_21.mT)], dim=-1) 

In [ ]:
M_2 = torch.cat([M_21, P_m_C.mT], dim=-1)

In [ ]:
M_1.shape, M_2.shape

(torch.Size([2, 3, 7]), torch.Size([2, 4, 7]))

In [ ]:
M = torch.cat([M_1, M_2], dim=-2)

In [ ]:
U = torch.linalg.qr(M).R

so $U \in \mathbb{R}^{(n+k) \times (n+k)}$ and the need to get the bottom part of size $k \times k$

In [ ]:
n_dim_state = P_m_C.shape[-1]
n_dim_obs = R_C.shape[-1]

In [ ]:
P_C = U[:, -n_dim_state-1:-1, -n_dim_state-1:-1]

In [ ]:
P_C @ P_C.mT

tensor([[[ 0.9810, -0.0111,  0.0350, -0.0053],
         [-0.0111,  0.6581,  0.0959, -0.0871],
         [ 0.0350,  0.0959,  0.3107, -0.0406],
         [-0.0053, -0.0871, -0.0406,  0.1817]],

        [[ 0.9810, -0.0111,  0.0350, -0.0053],
         [-0.0111,  0.6581,  0.0959, -0.0871],
         [ 0.0350,  0.0959,  0.3107, -0.0406],
         [-0.0053, -0.0871, -0.0406,  0.1817]]], dtype=torch.float64,
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
kSR.n_dim_obs

3

In [ ]:
H.shape

torch.Size([1, 3, 4])

In [ ]:
H.shape

torch.Size([1, 3, 4])

In [ ]:
#| export
def _filter(A, H, B,
            Q, R,
            b, d,
            m0, P0,
            obs, mask, control,
            cov_checker=CheckPosDef()
           ) ->Tuple[List, List, List, List]: # m_ms, P_ms, ms, Ps
    """Filter observations using kalman filter """
    n_timesteps = obs.shape[-2]
    bs = obs.shape[0]
    # lists are mutable so need to copy them
    m_ms, P_ms, ms, Ps = [[None for _ in range(n_timesteps)].copy() for _ in range(4)] 

    for t in range(n_timesteps):
        if t == 0:
            m_ms[t], P_ms[t] = torch.stack([m0]*bs).unsqueeze(-1), torch.stack([P0]*bs)
        else:
            m_ms[t], P_ms[t] = _filter_predict(A, Q, b, B,
                                                                      ms[t - 1], Ps[t - 1], control[:,t,:],
                                                                      cov_checker.add_args(t=t))

        ms[t], Ps[t] = _filter_correct(H, R, d,
                                                                     m_ms[t], P_ms[t],
                                                                     obs[:,t,:], mask[:,t,:],
                                                                     cov_checker.add_args(t=t))
    
    ret = list(maps(torch.stack, _times2batch, (m_ms, P_ms, ms, Ps,)))
    return ret

In [ ]:
obs, m0, P0 = data, k.m0, k.P0

In [ ]:
m_ms, P_ms, ms, Ps = _filter(
    A, H, B,
    Q, R,
    b, d,
    m0, P0,
    data, mask, control)

Predictions at time `0` for both batches

In [ ]:
show_as_row(list(map(Self.shape(), (m_ms, P_ms, ms, Ps,))))

In [ ]:
show_as_row(list(map(lambda x:x[0][0], (m_ms, P_ms, ms, Ps,))))

### KalmanFilter method

In [ ]:
#| export
@patch
def _filter_all(self: KalmanFilter, obs, mask, control
               ) ->Tuple[List, List, List, List]: # m_ms, P_ms, ms, Ps
    """ wrapper around `_filter`"""
    obs, mask = self._parse_obs(obs, mask)
    return _filter(
            self.A, self.H,
            self.B if self.use_control else torch.zeros_like(self.B),
            self.Q, self.R,
            self.b, self.d,
            self.m0, self.P0,
            obs, mask, control,
            self.cov_checker
        )

## Additional

### Constructors

#### Simple parameters

In [ ]:
#| export
@patch(cls_method=True)
def init_simple(cls: KalmanFilter,
                n_dim, # n_dim_obs and n_dim_state
                dtype=torch.float64):
    """Simplest version of kalman filter parameters"""
    return cls(
        A =     torch.eye(n_dim, dtype=dtype),
        b =        torch.zeros(n_dim, dtype=dtype),        
        Q =        torch.eye(n_dim, dtype=dtype),        
        H =       torch.eye(n_dim, dtype=dtype),
        d =          torch.zeros(n_dim, dtype=dtype),          
        R =          torch.eye(n_dim, dtype=dtype),            
        B =     torch.eye(n_dim, dtype=dtype),
        m0 =  torch.zeros(n_dim, dtype=dtype),        
        P0 =   torch.eye(n_dim, dtype=dtype),
    )

In [ ]:
KalmanFilter.init_simple(2).state_dict()

#### Local slope

Local slope models are an extentions of local level model that in the state variable keep track of also the slope

Given $n$ as the number of dimensions of the observations

The transition matrix (`A`) is:

$$A = \left[\begin{array}{cc}I & I \\ 0 & I\end{array}\right]$$

where:

- $I \in \mathbb{R}^{n \times n}$
- $A \in \mathbb{R}^{2n \times 2n}$

the state $x \in \mathbb{R}^{2N \times 1}$ where the upper half keep track of the level and the lower half of the slope. $A \in \mathbb{R}^2N \times 2N$

the observation matrix (`H`) is:

$$H = \left[\begin{array}{cc}I & 0 \end{array}\right]$$

For the multivariate case the 1 are replaced with an identiy matrix


assuming that the control has the same dimensions of the observations then if we are doing a local slope model we have $B \in \mathbb{R}^{state \times contr}$:
$$ B = \begin{bmatrix} -I & I \\ 0 & 0 \end{bmatrix}$$

In [ ]:
#| export
from torch import hstack, eye, vstack, ones, zeros, tensor
from functools import partial
from sklearn.decomposition import PCA

In [ ]:
#| exporti
def set_dtype(*args, dtype=torch.float64):
    return [partial(arg, dtype=dtype) for arg in args] 

eye, ones, zeros, tensor = set_dtype(eye, ones, zeros, tensor)

In [ ]:
#| export
# @delegates(KalmanFilter)
@patch(cls_method=True)
def init_local_slope_pca(cls: KalmanFilter,
                n_dim_obs, # n_dim_obs and n_dim_contr
                n_dim_state: int, # n_dim_state
                df_pca: pd.DataFrame|None = None, # dataframe for PCA init, None no PCA init
                **kwargs
            ):
    """Local Slope + PCA init"""
    if df_pca is not None:
        comp = PCA(n_dim_state).fit(df_pca).components_
        H = tensor(comp.T) # transform state -> obs
        B = tensor(comp) # transform obs -> state
    else:
        H, B = eye(n_dim_obs), eye(n_dim_obs)
        
    return cls(
        A =     vstack([hstack([eye(n_dim_state),                eye(n_dim_state)]),
                                   hstack([zeros(n_dim_state, n_dim_state), eye(n_dim_state)])]),
        b =        zeros(n_dim_state * 2),        
        Q =        eye(n_dim_state * 2)*.1,        
        H =       hstack([H, zeros(n_dim_obs, n_dim_state)]),
        d =          zeros(n_dim_obs),          
        R =          eye(n_dim_obs)*.01,            
        B =     vstack([hstack([-B,                  B]),
                                   hstack([ zeros(n_dim_state,n_dim_obs), zeros(n_dim_state, n_dim_obs)])]),
        m0 =  zeros(n_dim_state * 2),        
        P0 =   eye(n_dim_state * 2) * 3,
        **kwargs
    ) 

In [ ]:
KalmanFilter.init_local_slope_pca(2,2,pd.DataFrame([[1,2], [2,4]])).state_dict()

## Export

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()

## Other

### Filter Predict UD

#### UDU Decomposition

In [ ]:
Lc = torch.linalg.cholesky(Q)

In [ ]:
Lc

In [ ]:
D0 = torch.diag(Lc)

In [ ]:
D0

In [ ]:
D0.unsqueeze(0).shape

In [ ]:
L = Lc / D0.unsqueeze(0)
L

In [ ]:
D = D0.pow(2)
D

In [ ]:
L @ torch.diag(D) @ L.T - Q

can compute gradients

In [ ]:
L.sum().backward(retain_graph=True) 

Cannot used PyTorch LDL function, because it doesn't compute gradients, and in any case we don't need it! We can use just use the cholesky decomposition. Actually we may just use cholesky directly and $I$ for the diagonal ...

In [ ]:
A = to_posdef(torch.rand(2, 3,3))

In [ ]:
torch.diagonal(A, dim1=-2, dim2=-1).shape

In [ ]:
def udu_decomposition(A):
    L = torch.linalg.cholesky(A)
    D0 = torch.diagonal(L, dim1=-2, dim2=-1)
    U = L / D0.unsqueeze(-2)
    D = D0.pow(2)
    return U, D

In [ ]:
U, D = udu_decomposition(A)

In [ ]:
torch.diag_embed(D), D

In [ ]:
for _ in range(10):
    A = to_posdef(torch.rand(5, 10,10))
    U,D = udu_decomposition(A)
    test_close(A, U @ torch.diag_embed(D) @ U.mT)

In [ ]:
torch.diag(Q)

In [ ]:
udu_decomposition(torch.diag(torch.diag(Q)))

The first step is to decompose the Qariance matrix Q
$$    Q = GD_QG^T $$

In [ ]:
G, D_Q = udu_decomposition(Q)

$$W = \begin{bmatrix}AU_{t-1}&G\end{bmatrix}$$

In [ ]:
U_curr, D_curr = udu_decomposition(P_pr[0])

In [ ]:
W = torch.hstack([A @ U_curr, G])
W

$$ D_w = \begin{bmatrix}D_{t-1} & 0 \\ 0 & D_Q \end{bmatrix}$$

In [ ]:
Dw = torch.diag(torch.hstack([D_curr, D_Q]))
Dw

In [ ]:
W.shape

In [ ]:
C_dw = Dw.sqrt()

In [ ]:
C_dw.shape

In [ ]:
Q, R = torch.linalg.qr((W @ C_dw).mT)
Q, R

In [ ]:
Q.T @ Q

In [ ]:
U_pred = R.mT
V = Q.T @ torch.inverse(C_dw)
U_pred.shape, V.shape

In [ ]:
D_pred = V @ Dw @ V.T

In [ ]:
D_pred

so the math works and D_pred is a diagonal matrix, actually and identity matrix

Actually this all derivations are probably not needed, as the original idea of the algorightm is to not scalar square roots in PyTorch ... but since we have to use cholesky decomposition anyway, we can just use cholesky factors and make the life easier

In [ ]:
U_pred @ D_pred @ U_pred.T

In [ ]:
test_close(P_m[0], U_pred @ D_pred @ U_pred.T)

#| export
def filter_predict_ud(A,
                    Q,
                    b,
                    B, #[n_dim_state, n_dim_contr]
                    m_pr,
                    P_pr,
                    control, #[n_batches, n_dim_contr]
                    cov_checker=CheckPosDef()):
    
    